Nos han proporcionado un archivo xml para que abramos, leamos, modifiquemos e insertemos los datos en la tabla que tenemos creada en nuestra base de datos.

Vamos a crear una clase que recoja todos estos cambios que debemos realizar asi como la apertura del archivo.

Los cambios son eliminar la columna rebundante y modificar el genero para que aparezca como un string en vez de valor numerico 

In [13]:
import re
import os
import pandas as pd
import xml.etree.ElementTree as ET

#Creamos la clase que unira las funciones de lectura del archivo y limpieza

class Limpiar_xml:

    def __init__ (self,nombre_archivo):
        self.nombre_archivo = nombre_archivo

    def lectura_xml(self):  #abrimos el archivo 

        tree = ET.parse(self.nombre_archivo)
        root = tree.getroot()
        
        self.xml_lista = []

        for child in root:
            xml_dict = {}
            for subchild in child: 
                xml_dict[subchild.tag] = subchild.text
            self.xml_lista.append(xml_dict)
        return self.xml_lista

    def limpieza_xml (self, xml_lista):   #con esta funcion cambiamos los valores de la columna gender
        self.xml_lista = xml_lista
        for diccionario in xml_lista:

            if diccionario['gender'] == "0" :
                diccionario["gender"] = "Man"
            elif diccionario["gender"] == "1":
                diccionario["gender"] = "Woman"
            elif diccionario['gender'] == "2":
                diccionario['gender'] = "Nonbinary"
            elif diccionario['gender'] == "3":
                diccionario['gender'] = "Prefer not to say"
            elif diccionario['gender'] == "4":
                diccionario['gender'] = "Prefer to self-describe"
    
        for diccionario in xml_lista:   #y eliminamos la columna redundante
            diccionario.pop("level_0")

        return self.xml_lista

    def sacar_values(self, xml_lista):   #Cuando ya tenemos el archivo limpio con los cambios pedidos hacemos una lista de tuplas para poder introducir los datos en la base de datos
        self.xml_lista = xml_lista

        values = []
        for i in xml_lista:
            values.append(tuple(i.values()))
        return values

    



In [14]:
xml_clase = Limpiar_xml("data_xml.xml")

In [15]:
fichero_xml = xml_clase.lectura_xml()

In [16]:
xml_limpio = xml_clase.limpieza_xml(fichero_xml)

In [17]:
xml_tuplas = xml_clase.sacar_values(xml_limpio)

In [20]:
#Creamos una funcion para introducir los datos a la tabla correspondiente en nuestra base de datos

def importar_datos (xml_tuplas):
    import mysql.connector
    cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1', database='proyecto1')
    mycursor = cnx.cursor()

    sql = "INSERT INTO tabla_xml (index_xml,time,age,gender) VALUES (%s,%s,%s,%s)"
    val = xml_tuplas
    try:
        mycursor.executemany(sql, val)
        cnx.commit()
        print(mycursor.rowcount, "registro/s insertado/s.")
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [21]:
#llamamos a la funcion y ya tenemos los registros introducidos
importar_datos(xml_tuplas)

1062 (23000): Duplicate entry '1' for key 'tabla_xml.PRIMARY'
Error Code: 1062
SQLSTATE 23000
Message Duplicate entry '1' for key 'tabla_xml.PRIMARY'


Para poder ver mejor nuestra tabla, vamos a mostrarla con la libreria pandas

In [11]:
#Realizamos la conexion con la base de datos y la importacion de la libreria
import mysql.connector

from mysql.connector import errorcode

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='proyecto1')

# iniciamos el cursor
mycursor = cnx.cursor()

# ejecutamos nuestra query
mycursor.execute("SELECT * FROM tabla_xml")

# le decimos que nos devuelva todos los resultados y los almacenamos en una variable llamada myresult
myresult = mycursor.fetchall()

#Creamos un dataframe con los resultados de la consulta SQL almacenados en myresult. 
df = pd.DataFrame(myresult, columns = ['index_xml', 'time', 'age','gender','index_sql'])

#Cerramos la conexion
cnx.close()

KeyboardInterrupt: 

In [ ]:
#mostramos un ejemplo
df.head()